In [84]:
import pandas as pd
import json
from mplsoccer import Pitch
from datetime import datetime

DATA_FOLDER = "data"

# Get the matches

In [89]:
# Get all matches
matches_df = pd.read_parquet(f"{DATA_FOLDER}/matches.parquet")

# Create new column with match name (team names)
matches_df["match"] = (
    matches_df["home_team"].apply(lambda x: x["short_name"])
    + " vs "
    + matches_df["away_team"].apply(lambda x: x["short_name"])
)

# Create data column with match date
matches_df["date"] = pd.to_datetime(matches_df["date_time"]).dt.date

# Sort by date
matches_df = matches_df.sort_values('date_time', ascending=True)

matches_df.tail()

# Get games from 2024-2025 season
start_date_season_24_25 = datetime.strptime("2024-08-10", "%Y-%m-%d").date()
season_24_25_matches = matches_df[matches_df["date"] > start_date_season_24_25]

# Get games from 2023-2024 season
season_23_24_matches = matches_df[matches_df["date"] < start_date_season_24_25]

,id,date_time,home_team,away_team,status,competition_id,season_id,competition_edition_id,match,date
56,1528510,2024-05-11T16:30:00Z,"{'id': 283, 'short_name': 'Granada CF'}","{'id': 262, 'short_name': 'Real Madrid'}",closed,4,28,546,Granada CF vs Real Madrid,2024-05-11
55,1534700,2024-05-14T19:30:00Z,"{'id': 262, 'short_name': 'Real Madrid'}","{'id': 313, 'short_name': 'Alavés'}",closed,4,28,546,Real Madrid vs Alavés,2024-05-14
54,1543050,2024-05-19T17:00:00Z,"{'id': 260, 'short_name': 'Villarreal CF'}","{'id': 262, 'short_name': 'Real Madrid'}",closed,4,28,546,Villarreal CF vs Real Madrid,2024-05-19
53,1552423,2024-05-25T19:00:00Z,"{'id': 262, 'short_name': 'Real Madrid'}","{'id': 269, 'short_name': 'Betis Balompié'}",closed,4,28,546,Real Madrid vs Betis Balompié,2024-05-25
52,1562415,2024-06-01T19:00:00Z,"{'id': 103, 'short_name': 'Dortmund'}","{'id': 262, 'short_name': 'Real Madrid'}",closed,10,28,569,Dortmund vs Real Madrid,2024-06-01


# Get dynamic data for a certain match
For example: the last game in the dataset:
- id: 2018396
- match: Real Madrid vs Real Sociedad
- date: 2025-05-24

For a more detailed explanation: check the [Dynamic Events CSV Specifications file](./20250314%20-%20Dynamic%20Events%20CSV%20Specifications.pdf)

In [77]:
match_id = 2018396

# Get dynamic data for this match
def get_dynamic_data(match_id):
    dynamic_data = pd.read_parquet(f"{DATA_FOLDER}/dynamic/{match_id}.parquet")
    return dynamic_data

dynamic_data = get_dynamic_data(match_id)
dynamic_data.head(20)

,event_id,index,match_id,frame_start,frame_end,frame_physical_start,time_start,time_end,minute_start,second_start,...,xloss_player_possession_end,xloss_player_possession_max,xshot_player_possession_start,xshot_player_possession_end,xshot_player_possession_max,is_player_possession_start_matched,is_player_possession_end_matched,is_previous_pass_matched,is_pass_reception_matched,fully_extrapolated
0,8_0,0,2018396,20,20,NaN,00:01.0,00:01.0,0,1,...,NaN,NaN,NaN,NaN,NaN,False,False,None,False,True
1,7_0,1,2018396,20,20,NaN,00:01.0,00:01.0,0,1,...,NaN,NaN,NaN,NaN,NaN,False,False,None,False,True
2,7_1,2,2018396,20,20,NaN,00:01.0,00:01.0,0,1,...,NaN,NaN,NaN,NaN,NaN,False,False,None,None,True
3,8_1,3,2018396,28,35,NaN,00:01.8,00:02.5,0,1,...,NaN,NaN,NaN,NaN,NaN,False,False,False,True,True
4,7_2,4,2018396,28,35,NaN,00:01.8,00:02.5,0,1,...,NaN,NaN,NaN,NaN,NaN,False,False,None,True,True
5,7_3,5,2018396,28,35,NaN,00:01.8,00:02.5,0,1,...,NaN,NaN,NaN,NaN,NaN,False,False,None,None,True
6,8_2,6,2018396,40,64,NaN,00:03.0,00:05.4,0,3,...,NaN,NaN,NaN,NaN,NaN,True,True,False,True,False
7,7_4,7,2018396,40,42,NaN,00:03.0,00:03.2,0,3,...,NaN,NaN,NaN,NaN,NaN,True,True,None,None,True
8,7_5,8,2018396,40,49,NaN,00:03.0,00:03.9,0,3,...,NaN,NaN,NaN,NaN,NaN,True,True,None,None,False
9,7_6,9,2018396,40,64,NaN,00:03.0,00:05.4,0,3,...,NaN,NaN,NaN,NaN,NaN,True,True,None,None,False


In [11]:
# Only keep player_possession events
player_possession_df = dynamic_data[dynamic_data["event_type"] == "player_possession"]
player_possession_df.head()

,event_id,index,match_id,frame_start,frame_end,frame_physical_start,time_start,time_end,minute_start,second_start,...,xloss_player_possession_end,xloss_player_possession_max,xshot_player_possession_start,xshot_player_possession_end,xshot_player_possession_max,is_player_possession_start_matched,is_player_possession_end_matched,is_previous_pass_matched,is_pass_reception_matched,fully_extrapolated
0,8_0,0,2018396,20,20,NaN,00:01.0,00:01.0,0,1,...,NaN,NaN,NaN,NaN,NaN,False,False,None,False,True
3,8_1,3,2018396,28,35,NaN,00:01.8,00:02.5,0,1,...,NaN,NaN,NaN,NaN,NaN,False,False,False,True,True
6,8_2,6,2018396,40,64,NaN,00:03.0,00:05.4,0,3,...,NaN,NaN,NaN,NaN,NaN,True,True,False,True,False
14,8_3,14,2018396,80,90,NaN,00:07.0,00:08.0,0,7,...,NaN,NaN,NaN,NaN,NaN,True,True,True,True,False
19,8_4,19,2018396,97,97,NaN,00:08.7,00:08.7,0,8,...,NaN,NaN,NaN,NaN,NaN,True,True,True,True,False


# Exploring related columns

Some columns look pretty similar, let's look if they really are or what the differences are.

For example: does "channel_end" always have the same value as "player_targeted_channel_pass"? Maybe there's a difference between passes that are successful and unsuccessful?

In [17]:
rel_cols = [
    "index", "match_id", "team_shortname", 
    "phase_index",
    "pass_outcome",
    "channel_start", "channel_end", "player_targeted_channel_pass",
    "third_end", "player_targeted_third_pass",
    "penalty_area_end", "player_targeted_penalty_area_pass",
    "game_interruption_after", "pass_direction_received",
    "pass_ahead", "dangerous",
]

rel_cols_df = player_possession_df[rel_cols]
rel_cols_df.head()

,index,match_id,team_shortname,phase_index,pass_outcome,channel_start,channel_end,player_targeted_channel_pass,third_end,player_targeted_third_pass,penalty_area_end,player_targeted_penalty_area_pass,game_interruption_after,pass_direction_received,pass_ahead,dangerous
0,0,2018396,Real Madrid,0,successful,center,center,center,middle_third,middle_third,False,False,None,None,False,None
3,3,2018396,Real Madrid,0,successful,center,center,half_space_left,middle_third,middle_third,False,False,None,backward,False,None
6,6,2018396,Real Madrid,0,successful,half_space_left,half_space_left,center,middle_third,middle_third,False,False,None,sideway_left,True,None
14,14,2018396,Real Madrid,0,successful,half_space_right,half_space_right,half_space_right,middle_third,middle_third,False,False,None,sideway_right,True,None
19,19,2018396,Real Madrid,0,successful,half_space_right,half_space_right,half_space_right,middle_third,middle_third,False,False,None,forward,False,None


The difference between "channel_end" and "player_targeted_channel_pass" is that the "channel_end" column shows where the player of this possession ends up and that the "player_targeted" columns show where his intended pass will end up.

I always think it's a bit of a stretch to say what a player intended as only the players themselves will know that. In some cases it's pretty obvious but in (a lot of) other cases it can be up for debate. I'm not very comfortable integrating these columns as I don't feel their values can be objectively set.

# Keep relevant columns for possession chain

In [23]:
pos_chain_cols = [
    "index", "match_id", "team_shortname",
    "phase_index",
    "channel_start", "channel_end", 
    "third_start", "third_end", 
    "penalty_area_start", "penalty_area_end",
    "game_interruption_after", "pass_direction_received", "pass_outcome",
    "lead_to_shot", "lead_to_goal", "possession_danger",
    "pass_ahead", "dangerous",
    "first_line_break", "second_last_line_break", "last_line_break", "furthest_line_break",
    "n_opponents_bypassed"
]

pos_chain_df = player_possession_df[pos_chain_cols]
pos_chain_df.head()

,index,match_id,team_shortname,phase_index,channel_start,channel_end,third_start,third_end,penalty_area_start,penalty_area_end,...,lead_to_shot,lead_to_goal,possession_danger,pass_ahead,dangerous,first_line_break,second_last_line_break,last_line_break,furthest_line_break,n_opponents_bypassed
0,0,2018396,Real Madrid,0,center,center,middle_third,middle_third,False,False,...,False,False,None,False,None,False,False,False,None,-3.0
3,3,2018396,Real Madrid,0,center,center,middle_third,middle_third,False,False,...,False,False,None,False,None,False,False,False,None,-1.0
6,6,2018396,Real Madrid,0,half_space_left,half_space_left,middle_third,middle_third,False,False,...,False,False,None,True,None,False,False,False,None,0.0
14,14,2018396,Real Madrid,0,half_space_right,half_space_right,middle_third,middle_third,False,False,...,False,False,None,True,None,True,False,False,first,2.0
19,19,2018396,Real Madrid,0,half_space_right,half_space_right,middle_third,middle_third,False,False,...,False,False,None,False,None,False,False,False,None,-2.0


In [28]:
pos_chain = pos_chain_df[pos_chain_df["phase_index"] == 0]
pos_chain

,index,match_id,team_shortname,phase_index,channel_start,channel_end,third_start,third_end,penalty_area_start,penalty_area_end,...,lead_to_shot,lead_to_goal,possession_danger,pass_ahead,dangerous,first_line_break,second_last_line_break,last_line_break,furthest_line_break,n_opponents_bypassed
0,0,2018396,Real Madrid,0,center,center,middle_third,middle_third,False,False,...,False,False,None,False,None,False,False,False,None,-3.0
3,3,2018396,Real Madrid,0,center,center,middle_third,middle_third,False,False,...,False,False,None,False,None,False,False,False,None,-1.0
6,6,2018396,Real Madrid,0,half_space_left,half_space_left,middle_third,middle_third,False,False,...,False,False,None,True,None,False,False,False,None,0.0
14,14,2018396,Real Madrid,0,half_space_right,half_space_right,middle_third,middle_third,False,False,...,False,False,None,True,None,True,False,False,first,2.0
19,19,2018396,Real Madrid,0,half_space_right,half_space_right,middle_third,middle_third,False,False,...,False,False,None,False,None,False,False,False,None,-2.0
23,23,2018396,Real Madrid,0,half_space_right,half_space_right,middle_third,middle_third,False,False,...,False,False,None,False,None,False,False,False,None,0.0
26,26,2018396,Real Madrid,0,center,center,defensive_third,defensive_third,False,False,...,False,False,None,False,None,False,False,False,None,0.0
33,33,2018396,Real Madrid,0,half_space_right,wide_right,defensive_third,middle_third,False,False,...,False,False,None,False,None,False,False,False,None,0.0
41,41,2018396,Real Madrid,0,wide_right,wide_right,middle_third,middle_third,False,False,...,False,False,None,False,None,False,False,False,None,0.0
43,43,2018396,Real Madrid,0,half_space_right,center,defensive_third,defensive_third,False,False,...,False,False,None,False,None,False,False,False,None,0.0


# Score parts of possession chain

In [60]:
# third_end == "attacking_third"
# penalty_area_end = True
# game_interruption_after == "corner_for", "free_kick_for", "goal_for", "penalty_for", "throw_in_for"
# pass_direction_received == "forward"
# pass_outcome == "successful"
# lead_to_shot == True
# lead_to_goal == True
# possession_danger == True
# pass_ahead == True
# dangerous == True
# first_line_break == True
# second_last_line_break == True
# last_line_break == True
# furthest_line_break == True
# n_opponents_bypassed == (int)

def score_possession_chain(chain_df):
    # Init channel scores
    channel_scores = {}

    # Loop through events of current chain
    for index, row in chain_df.iterrows():
        start_channel = row["channel_start"]
        end_channel = row["channel_end"]

        # Add 1 point to start channel
        if start_channel in channel_scores:
            channel_scores[start_channel] += 1
        else:
            channel_scores[start_channel] = 1

        # Add 1 point to end channel
        if end_channel in channel_scores:
            channel_scores[end_channel] += 1
        else:
            channel_scores[end_channel] = 1

        # Calculate total points
        total_points = sum(channel_scores.values())

        # Calculate percentages
        channel_percentages = {channel: (points / total_points) * 100 for channel, points in channel_scores.items()}

        # Calculate wing and central scores/percentages
        wing_channels = ["wide_right", "wide_left"]
        central_channels = ["center"]

        wing_score = sum(channel_scores.get(channel, 0) for channel in wing_channels)
        central_score = sum(channel_scores.get(channel, 0) for channel in central_channels)

        # Adjust scores with half space channels
        half_space_channels = ["half_space_right", "half_space_left"]
        half_space_score = sum(channel_scores.get(channel, 0) for channel in half_space_channels)
        wing_score += half_space_score / 2
        central_score += half_space_score / 2

        wing_percentage = wing_score / total_points
        central_percentage = central_score / total_points

        # Classification logic
        if wing_percentage > 0.5:
            classification = "wing"
        elif central_percentage > 0.5:
            classification = "central"
        else:
            classification = "mixed"

    return {
        'channel_scores': channel_scores,
        'channel_percentages': channel_percentages,
        'wing_score': wing_score,
        'central_score': central_score,
        'total_points': total_points,
        'wing_percentage': wing_percentage,
        'central_percentage': central_percentage,
        'classification': classification
    }

score_possession_chain(pos_chain)

{'channel_scores': {'center': 11,
  'half_space_left': 2,
  'half_space_right': 13,
  'wide_right': 4,
  'wide_left': 2},
 'channel_percentages': {'center': 34.375,
  'half_space_left': 6.25,
  'half_space_right': 40.625,
  'wide_right': 12.5,
  'wide_left': 6.25},
 'wing_score': 13.5,
 'central_score': 18.5,
 'total_points': 32,
 'wing_percentage': 0.421875,
 'central_percentage': 0.578125,
 'classification': 'central'}

# Analyze all possession chains in a game

In [61]:
def analyze_match_chains(data):
    madrid_chains = data[data["team_shortname"] == "Real Madrid"]["phase_index"].unique()
    chain_results = []

    for chain_id in madrid_chains:
        chain_df = data[data["phase_index"] == chain_id]
        chain_result = score_possession_chain(chain_df)

        # Chains have to be at least 3 events long
        chain_length = chain_df.shape[0]
        if chain_length < 3:
            continue

        # Add chain data to result
        chain_results.append({
            "match_id": chain_df["match_id"].iloc[0],
            "chain_id": chain_id,
            "chain_length": chain_length,
            "classification": chain_result["classification"],
            "wing_percentage": chain_result["wing_percentage"],
            "central_percentage": chain_result["central_percentage"]
        })

    return pd.DataFrame(chain_results)


test = analyze_match_chains(pos_chain_df)
test

,match_id,chain_id,chain_length,classification,wing_percentage,central_percentage
0,2018396,0,16,central,0.421875,0.578125
1,2018396,2,4,wing,0.562500,0.437500
2,2018396,9,4,wing,0.625000,0.375000
3,2018396,12,8,central,0.250000,0.750000
4,2018396,15,9,wing,0.611111,0.388889
...,...,...,...,...,...,...
86,2018396,298,7,wing,0.750000,0.250000
87,2018396,301,3,mixed,0.500000,0.500000
88,2018396,303,6,wing,0.708333,0.291667
89,2018396,307,3,central,0.333333,0.666667


In [71]:
mean_wing = test['wing_percentage'].mean()
mean_central = test['central_percentage'].mean()
print(f"\nAverage wing play: {mean_wing:.2f}%")
print(f"Average central play: {mean_central:.2f}%")


Average wing play: 0.51%
Average central play: 0.49%


In [70]:
chain_types = test['classification'].value_counts(normalize=True)
print("Proportion of each chain type:")
print(chain_types)

Proportion of each chain type:
classification
wing       0.494505
central    0.428571
mixed      0.076923
Name: proportion, dtype: float64


# Analyzing whole seasons

In [ ]:
# all games = matches_df
